# TensorFlow Intro

* Names and execution in Python and TensorFlow
* The simplest TensorFlow neuron
* Making the neuron learn
* Training diagnostics in TensorBoard
* Flowing onward

*courtesy of [Aaron Schumacher](https://www.oreilly.com/learning/hello-tensorflow)*

### Names and execution in Python and TensorFlow
#### A name in Python is separated from the object. A name points at the object and not the other way around.


<img src="img/an_object_has_no_name.jpg" alt="An object has no name." style="width: 400px;"/>



*Image courtesy of [Hadley Wickham](https://twitter.com/hadleywickham/status/732288980549390336).*

Let's see a code example of foo pointing to an object:

In [1]:
foo = []

In [2]:
bar = foo

In [3]:
foo == bar

True

In [4]:
foo is bar

True

In [5]:
id(foo)

36658424

In [6]:
id(bar)

36658424

You can now see that id(foo) and id(bar) are the same.

In [7]:
# Nesting lists is one way to represent a graph structure like a TensorFlow computation graph.
foo.append(bar) 


In [8]:
# and now foo is inside itself...
foo

[[...]]

Python is managing foo and bar and both names point at the same thing.

<img src="img/foo_loop.png" alt="foo points to itself" style="width: 400px;"/>

*Image made with [draw.io](https://draw.io/).*


Meaning that...
### Tensorflow has also a system to keep track of things somewhere and giving access via names.


### Let's start by the simplest TensorFlow neuron
*Will just have an input and a weight and we will be able to multiply that*

In [9]:
# Inport tensorflow, create the graph and define the values
import tensorflow as tf
sess = tf.Session()
graph = tf.get_default_graph()
input_value = tf.constant(1.0)
weight = tf.Variable(0.8)

In [10]:
# We define our operation in the graph
output_value = weight * input_value
op = graph.get_operations()[-1]

### What is the last operation in the graph? Multiplication.

In [11]:
op.name

u'mul'

In [12]:
# Generates an operation which will initialize all our variables a finally run it

sess.run(tf.global_variables_initializer())
sess.run(output_value)

0.80000001

*Recall that's 0.8 x 1.0 with 32-bit floats, and 32-bit floats have a hard time with 0.8; 0.80000001 is as close as they can get.*

This is the neuron's "inference" or "forward pass".

### Neat! let's look at our graph in TensorBoard

In [13]:
# reset previous stuff
tf.reset_default_graph()
sess = tf.Session()
!rm -rf log_simple_graph
!rm -rf log_simple_stat

x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.multiply(w, x, name='output')

In [14]:
# FileWriter needs an output directory.
summary_writer = tf.summary.FileWriter('log_simple_graph', sess.graph)

In [15]:
!tensorboard --logdir=log_simple_graph

TensorBoard 0.1.5 at http://ricardo-T440s:6006 (Press CTRL+C to quit) ^C



To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing `esc`, `i`, `i`.

### Making the neuron learn with gradient descent
Now the system takes the input one and returns 0.8, which is worg. We need a way to measure how wrong the system is. We'll call that measure the “loss” and give our system the goal of minimizing the loss. 


In [16]:
y_ = tf.constant(0.0)

We make the loss the **square of the difference** between the **current output** and the **desired output**, so it doesn't get to a negative values.

In [17]:
loss = (y - y_)**2

# Optimizer with a learning rate of 0.025.
optim = tf.train.GradientDescentOptimizer(learning_rate=0.025)

<img src="img/plot.png" alt="plot of loss and derivative." style="width: 400px;"/>
<center>Plot of Loss and Derivative</center>

In [18]:
grads_and_vars = optim.compute_gradients(loss)
sess.run(tf.global_variables_initializer())
sess.run(grads_and_vars[0][0])

1.6

Why is the value of the gradient 1.6? Our loss is error squared, and the derivative of that is two times the error. Currently the system says 0.8 instead of 0, so the error is 0.8, and two times 0.8 is 1.6. It's working!

We have `y = 0.8` and `y_ = 0`.

So the loss is `(0.8 - 0)^2 = 0.64`.

And the derivative of the loss is `2*(0.8 - 0)`.

#### Let's apply the gradient, finishing the backpropagation.

In [19]:
sess.run(optim.apply_gradients(grads_and_vars))
sess.run(w)

0.75999999

The weight decreased by 0.04 because the optimizer subtracted the gradient times the learning rate, 1.6 * 0.025, pushing the weight in the right direction.

`0.8 - 0.025*1.6 = 0.8 - 0.04 = 0.76`

Instead of hand-holding the optimizer like this, we can make one operation that calculates and applies the gradients: the `train_step`

In [20]:
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

for i in range(100):
    sess.run(train_step)

sess.run(y)

0.0044996012

Running the training step many times, the weight and the output value are now very close to zero. 

The neuron has learned!


# Last Step: The all example on Tensorboard

In [25]:
# reset everything 
tf.reset_default_graph()
!rm -rf log_simple_stats

The script below is designed to stand alone, and will also work here with the default graph reset.

In [26]:
import tensorflow as tf

x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.multiply(w, x, name='output')
y_ = tf.constant(0.0, name='correct_value')
loss = tf.pow(y - y_, 2, name='loss')
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

for value in [x, w, y, y_, loss]:
    tf.summary.scalar(value.op.name, value)

summaries = tf.summary.merge_all()

session = tf.Session()
summary_writer = tf.summary.FileWriter('log_simple_stats', session.graph)

session.run(tf.global_variables_initializer())
for i in range(100):
    summary_writer.add_summary(session.run(summaries), i)
    session.run(train_step)

summary_writer.close()

In [27]:
!tensorboard --logdir=log_simple_stats

TensorBoard 0.1.5 at http://ricardo-T440s:6006 (Press CTRL+C to quit) ^C



After startup, go to http://localhost:6006/#events to see the interface.

To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing `esc`, `i`, `i`.

### Next Demo: http://localhost:8888/notebooks/final/ImageClassifier_GPU.ipynb 
